# 🔎 特徴量分析（AutoViz + PyCaret）v1 — AI売上予測ツール

## 📘 このツールの目的
**AIが売上データを自動分析し、「何が売上に効くか」を教えてくれるツールです。**

### 🎯 できること
1. グラフで視覚的に売上パターンを確認（AutoViz）
2. AI が重要な要因を自動で見つける（PyCaret）
3. 予測モデルを作成し、精度を数値で確認

---

## 🚀 使い方（簡単3ステップ）
1. **セルを上から順に実行**（各セルで Shift+Enter を押す）
2. **グラフを見て、パターンを確認**
3. **重要度ランキングで、注目すべき要因を特定**

---

**それでは、以下のセルを順番に実行してください。各セクションに「何を見るか」「どう判断するか」が書いてあります。**

In [ ]:
# ライブラリと環境
import warnings; warnings.filterwarnings('ignore')
import os, sys
from pathlib import Path
import pandas as pd, numpy as np

# 日本語フォント（Matplotlib/Plotly）
import font_setup  # IPAGothic等を自動設定

# AutoViz
try:
    from autoviz.AutoViz_Class import AutoViz_Class
    AV_OK = True
except Exception:
    AV_OK = False

# PyCaret（回帰）
try:
    from pycaret.regression import setup, compare_models, pull, finalize_model, predict_model, plot_model
    PYC_OK = True
except Exception:
    PYC_OK = False

# ipywidgets（インタラクティブUI）
WIDGETS = False
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    WIDGETS = True
except Exception:
    pass

# バージョン情報
print(f'Python: {sys.version.split()[0]}')
print(f'pandas: {pd.__version__}, numpy: {np.__version__}')
print(f'AutoViz: {AV_OK}, PyCaret.regression: {PYC_OK}')

# PyCaretの最小サンプル数（3-fold CVに必要）
MIN_SAMPLES_PYCARET = 100

# ========================================
# 🚀 GPU検出と設定
# ========================================
print('\n' + '='*60)
print('🖥️ GPU検出')
print('='*60)

# GPU利用可能性をチェック
GPU_AVAILABLE = False
GPU_DEVICE = 'cpu'

try:
    import torch
    if torch.cuda.is_available():
        GPU_AVAILABLE = True
        GPU_DEVICE = 'cuda'
        print(f'✅ NVIDIA GPU検出: {torch.cuda.get_device_name(0)}')
        print(f'   CUDA Version: {torch.version.cuda}')
        print(f'   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB')
    else:
        print('⚠️ PyTorch installed but no CUDA GPU found')
except ImportError:
    print('ℹ️ PyTorch not installed (GPU detection skipped)')

# LightGBM GPU対応チェック
try:
    import lightgbm as lgb
    if GPU_AVAILABLE:
        print('✅ LightGBM GPU対応: 可能')
    else:
        print('ℹ️  LightGBM: CPU mode')
except ImportError:
    print('ℹ️ LightGBM not installed')

# XGBoost GPU対応チェック
try:
    import xgboost as xgb
    if GPU_AVAILABLE:
        print('✅ XGBoost GPU対応: 可能')
    else:
        print('ℹ️ XGBoost: CPU mode')
except ImportError:
    print('ℹ️ XGBoost not installed')

# CatBoost GPU対応チェック
try:
    import catboost
    if GPU_AVAILABLE:
        print('✅ CatBoost GPU対応: 可能')
    else:
        print('ℹ️ CatBoost: CPU mode')
except ImportError:
    print('ℹ️ CatBoost not installed')
# cuDF (GPU Pandas) チェック
CUDF_AVAILABLE = False
try:
    import cudf
    if GPU_AVAILABLE:
        CUDF_AVAILABLE = True
        print('✅ cuDF (GPU Pandas) 対応: 可能')
    else:
        print('ℹ️ cuDF: GPU not available, using CPU Pandas')
except ImportError:
    print('ℹ️ cuDF not installed (CPU Pandas継続)')

# GPU使用フラグ（ユーザーが変更可能）
USE_GPU = GPU_AVAILABLE  # Trueに設定するとGPUを使用（GPUが利用可能な場合のみ）

if USE_GPU:
    print(f'\n🚀 GPU使用: 有効（推定2～10倍高速化）')
    print(f'   Device: {GPU_DEVICE}')
else:
    print(f'\nℹ️ GPU使用: 無効（CPUモードで実行）')
    if GPU_AVAILABLE:
        print('   💡 ヒント: 上のセルで USE_GPU = True に設定するとGPU使用可能')

print('='*60)


In [ ]:
# ========================================
# 🔧 GPU/CPU データフレーム変換ユーティリティ
# ========================================

def to_gpu(df):
    """pandasデータフレームをGPU (cuDF) に変換（USE_GPU=Trueの場合のみ）"""
    if USE_GPU and CUDF_AVAILABLE and df is not None and not df.empty:
        try:
            import cudf
            return cudf.from_pandas(df)
        except Exception as e:
            print(f'⚠️ GPU変換失敗、CPUモード継続: {e}')
            return df
    return df

def to_cpu(df):
    """cuDFデータフレームをpandasに変換"""
    if df is None:
        return None
    try:
        import cudf
        if isinstance(df, cudf.DataFrame):
            return df.to_pandas()
    except:
        pass
    return df

# GPUメモリ使用状況表示
def show_gpu_memory():
    if GPU_AVAILABLE:
        try:
            import torch
            allocated = torch.cuda.memory_allocated(0) / 1024**3
            reserved = torch.cuda.memory_reserved(0) / 1024**3
            print(f'📊 GPU Memory: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved')
        except:
            pass

if USE_GPU and CUDF_AVAILABLE:
    print('✅ GPU高速化ユーティリティ: 準備完了')
    print('   to_gpu(df) でGPU処理、to_cpu(df) でCPU戻し')
    show_gpu_memory()
else:
    print('ℹ️ CPU処理モード（GPUユーティリティは無効）')

In [ ]:

# フォント強制リセット（日本語対応 / AutoViz対策）
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
try:
    import japanize_matplotlib; japanize_matplotlib.japanize()
except Exception:
    pass
candidates = ['IPAGothic','IPAexGothic','Noto Sans CJK JP','Noto Sans JP','Yu Gothic','Meiryo','Hiragino Sans','MS Gothic']
avail = []
try:
    names = [getattr(f,'name','') for f in fm.fontManager.ttflist]
    for nm in candidates:
        if any(nm in n for n in names):
            if nm not in avail:
                avail.append(nm)
except Exception:
    pass
if not avail:
    avail = ['Noto Sans CJK JP','IPAGothic','DejaVu Sans']
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = avail + ['DejaVu Sans']
plt.rcParams['font.serif'] = ['Noto Serif CJK JP','IPAMincho','DejaVu Serif']
plt.rcParams['axes.unicode_minus'] = False
try:
    sns.set_theme(rc={'font.family':'sans-serif','font.sans-serif': plt.rcParams['font.sans-serif']})
except Exception:
    pass
print('フォント設定:', plt.rcParams['font.family'], '→', plt.rcParams['font.sans-serif'][:3], '...')

## 📂 ステップ1: データ読み込み

### 🎯 **このセクションの目的**
売上データ（CSV形式）を自動検出して読み込みます。

### 👀 **店長が確認すべきこと**
実行後、以下のメッセージが表示されます：
```
[INFO] glob検出: output/06_final_enriched_20250701_20250930.csv
[INFO] 列名変換: 8/8 件適用 → ['store_id', 'sku_id', 'date'...]
読み込み完了: 06_final_enriched_20250930.csv | shape=(50000, 40) | memory=15.2MB
```

**意味**:
- `shape=(50000, 40)` → 50,000行×40列のデータ
- `memory=15.2MB` → メモリ使用量

**判断ポイント**:
- データが見つからない場合 → `output`フォルダにCSVファイルがあるか確認
- メモリ不足エラーが出る場合 → データ期間を短縮検討

---

**次のセルを実行してください ↓**

In [ ]:
def pick_enriched_csv():
    """
    enriched CSVファイルを自動検出
    優先順: 環境変数 → glob最新 → 既定値
    """
    for env in ('DATA_PATH','ENRICHED_CSV'):
        p = os.environ.get(env)
        if p and Path(p).exists():
            print(f'[INFO] 環境変数 {env} からファイル取得: {p}')
            return Path(p)
    d = Path('output')
    if d.exists():
        c = sorted(d.glob('06_final_enriched_*.csv'), reverse=True)
        if c:
            print(f'[INFO] glob検出: {c[0]}')
            return c[0]
    f = Path('output/06_final_enriched_20250701_20250930.csv')
    if f.exists():
        print(f'[INFO] 既定ファイル使用: {f}')
        return f
    return None

CSV_PATH = pick_enriched_csv()
if not CSV_PATH:
    raise FileNotFoundError('output/06_final_enriched_*.csv が見つかりません。work/output に配置、または環境変数 DATA_PATH で指定してください。')

# メモリチェック（オプション）
try:
    import psutil
    file_size_mb = CSV_PATH.stat().st_size / (1024**2)
    available_mem_mb = psutil.virtual_memory().available / (1024**2)
    if file_size_mb > available_mem_mb * 0.5:
        print(f'[WARNING] ファイルサイズ {file_size_mb:.1f}MB が利用可能メモリの50%超')
except ImportError:
    pass

df_raw = pd.read_csv(CSV_PATH, encoding='utf-8-sig')
df = df_raw.copy()

# 列名正規化（存在すれば）
rename_map = {
    '店舗':'店舗','商品名':'商品名','日付':'日付','売上数量':'売上数量','売上金額':'売上金額',
    'フェイスくくり大分類':'フェイスくくり大分類','フェイスくくり中分類':'フェイスくくり中分類','フェイスくくり小分類':'フェイスくくり小分類'
}
renamed = {k:v for k,v in rename_map.items() if k in df.columns}
df = df.rename(columns=renamed)
print(f'[INFO] 列名変換: {len(renamed)}/{len(rename_map)} 件適用 → {list(renamed.values())}')

if '日付' in df.columns: df['日付'] = pd.to_datetime(df['日付'])
if '売上数量' in df.columns: df['売上数量'] = pd.to_numeric(df['売上数量'], errors='coerce').fillna(0).astype('float32')
if '売上金額' in df.columns: df['売上金額'] = pd.to_numeric(df['売上金額'], errors='coerce').fillna(0).astype('float32')

print(f'読み込み完了: {CSV_PATH.name} | shape={df.shape} | memory={df.memory_usage(deep=True).sum()/(1024**2):.1f}MB')
df.head(3)

## 📊 ステップ2: 売上最大化のためのフラグ別売上分析

### 🎯 **このセクションの目的**
**売上を最大化するために、どのフラグ（天候・曜日・イベント等）の日に売上が伸びるかを分析します。**

### ⚠️ **重要な変更点（従来のAutoVizから変更）**
- ❌ **削除**: 日付ベースの時系列プロット（日付と売上の比較）
- ✅ **追加**: フラグ別の売上比較（降雨フラグ、週末フラグ等と売上の相関）
- ✅ **追加**: 商品カテゴリ選択機能（特定カテゴリに絞った分析が可能）

### 📈 **何が分析されるか**

#### 1️⃣ **フラグ別売上増加率ランキング**
- 各フラグ（降雨、週末、猛暑日など）がONの日とOFFの日で、売上がどれだけ変わるかを計算
- 売上増加率が高いフラグ = 売上最大化のチャンス日

#### 2️⃣ **フラグ別売上分布の可視化**
- フラグON時とOFF時の売上分布を比較
- どのフラグの日に高額売上が発生しやすいかを視覚的に把握

#### 3️⃣ **カテゴリ別分析**
- 商品カテゴリを選択して、カテゴリごとの売上パターンを分析
- 例: 「飲料」カテゴリは猛暑日に売上↑、「おでん」は冬日に売上↑

### 👀 **店長が確認すべきこと**

#### ✅ **実行後に表示される情報**
1. **フラグ別売上増加率 TOP10 テーブル**
   - 各フラグのON/OFF時の平均売上
   - 売上増加率（%）
   - 該当日数

2. **フラグ別売上増加率の棒グラフ**
   - 赤い棒（プラス）: このフラグの日は売上が増加 → **陳列・発注を強化すべき日**
   - 青い棒（マイナス）: このフラグの日は売上が減少 → 在庫を抑制

3. **上位5フラグのON/OFF売上分布グラフ**
   - 赤い分布（フラグON）が右にシフト → そのフラグの日は高額売上が発生しやすい
   - 青い分布（フラグOFF）との差が大きい → そのフラグの影響が大きい

### 💡 **店長の実務アクション**

#### 🎯 **売上増加率が高いフラグが見つかった場合**

**例: 「降雨フラグ」の売上増加率 +25%**
→ **アクション**:
- 降雨予報の日の前日に、温かい総菜・カップ麺・ホット飲料の発注を1.3倍に増やす
- 入口付近に傘・レインコート・ホット商品の特設コーナーを設置
- 中華まん・おでんのフェースを拡大

**例: 「週末フラグ」の売上増加率 +18%**
→ **アクション**:
- 金曜夕方から弁当・デザート・酒類の陳列を強化
- 土日の朝は朝食需要（パン・コーヒー）、昼は弁当、夕方は酒類のピーク対応
- 家族向け大容量商品のフェース拡大

**例: 「給料日フラグ」の売上増加率 +12%**
→ **アクション**:
- 給料日（25日前後）は高単価弁当・スイーツ・プレミアム商品を目立つ位置に
- 夕方の前出し時間を早める（17時→16時30分）

#### 📊 **カテゴリ選択の活用方法**

上のウィジェット（SelectMultiple）で商品カテゴリを選択すると、そのカテゴリに特化したフラグ分析が再実行されます。

**使い方**:
1. ウィジェットから分析したいカテゴリを選択（Ctrl/Cmd + クリックで複数選択）
2. 自動で再分析が実行され、そのカテゴリの売上最大化フラグが表示される
3. カテゴリごとの最適な発注・陳列戦略を立てる

**例**:
- **「飲料」**を選択 → 「猛暑日」「真夏日」フラグで売上↑ → 夏日の冷飲料発注強化
- **「弁当」**を選択 → 「週末フラグ」「昼ピーク」で売上↑ → 土日の弁当発注1.5倍
- **「デザート」**を選択 → 「給料日」「週末」で売上↑ → 高単価スイーツ陳列強化

### 🚀 **実行方法**
下のセルを実行してください。自動で以下が行われます:
1. カテゴリ選択ウィジェットの表示（複数選択可）
2. 全カテゴリでのフラグ別売上分析
3. TOP10フラグの表示とグラフ可視化
4. カテゴリ選択時の自動再分析

### 💾 **GPU高速化について**
- このセルでは主にグラフ描画を行うため、GPU効果は限定的です
- 次のステップ（PyCaret）で大規模なAI学習が実行され、GPUの効果が最大化されます

In [ ]:
# ========================================
# 📊 売上最大化分析: フラグ別売上比較（全カテゴリ）
# ========================================
print('\n' + '='*60)
print('📊 売上最大化分析: フラグ別売上比較')
print('='*60)

# カテゴリフィルタ（オプション：コメントアウトを外して使用）
# selected_categories = ['全カテゴリ']  # または ['菓子', '飲料'] のように指定
selected_categories = ['全カテゴリ']

def analyze_sales_by_flags():
    """フラグ別売上分析の実行"""
    
    # データ存在確認
    if 'df' not in globals():
        print('⚠️ データフレーム "df" が見つかりません。先にステップ1でデータを読み込んでください。')
        return
    
    # カテゴリフィルタ適用
    df_filtered = df.copy()
    if '全カテゴリ' not in selected_categories and 'フェイスくくり大分類' in df.columns:
        df_filtered = df_filtered[df_filtered['フェイスくくり大分類'].isin(selected_categories)]
        print(f'✅ フィルタ適用: {selected_categories} ({len(df_filtered):,}件)')
    else:
        print('ℹ️ 全カテゴリで分析します')
    
    # デバッグ: カラム名を表示
    print(f'\n📋 利用可能なカラム数: {len(df_filtered.columns)}')
    flag_like_cols = [c for c in df_filtered.columns if 'flag' in c.lower() or c.startswith('is_') or c.startswith('has_')]
    
    if flag_like_cols:
        print(f'📌 フラグ系カラム候補: {len(flag_like_cols)}個')
        for i, col in enumerate(flag_like_cols[:20], 1):  # 最初の20個まで表示
            print(f'  {i}. {col}')
        if len(flag_like_cols) > 20:
            print(f'  ... 他 {len(flag_like_cols)-20}個')
    
    # フラグカラム検出（柔軟な検索）
    flag_cols = []
    
    # パターン1: flag_ プレフィックス
    flag_cols.extend([c for c in df_filtered.columns if c.startswith('flag_')])
    
    # パターン2: is_ プレフィックス（ブール値）
    flag_cols.extend([c for c in df_filtered.columns if c.startswith('is_')])
    
    # パターン3: has_ プレフィックス
    flag_cols.extend([c for c in df_filtered.columns if c.startswith('has_')])
    
    # パターン4: 日本語フラグ（_フラグ で終わる）
    flag_cols.extend([c for c in df_filtered.columns if c.endswith('_フラグ') or c.endswith('フラグ')])
    
    # 重複削除
    flag_cols = list(set(flag_cols))
    
    if not flag_cols:
        print('\n⚠️ フラグカラムが見つかりません')
        print('\n💡 以下のいずれかの形式でフラグカラムを作成してください:')
        print('  - flag_雨天日')
        print('  - is_weekend')
        print('  - has_event')
        print('  - イベント_フラグ')
        print('\n📌 現在のカラムで数値型（0/1の候補）:')
        numeric_binary = []
        for col in df_filtered.select_dtypes(include=[np.number]).columns:
            unique_vals = df_filtered[col].dropna().unique()
            if len(unique_vals) == 2 and set(unique_vals).issubset({0, 1, 0.0, 1.0}):
                numeric_binary.append(col)
        
        if numeric_binary:
            for i, col in enumerate(numeric_binary[:10], 1):
                print(f'  {i}. {col}')
            print('\n💡 これらのカラムを "flag_" プレフィックス付きに変更すると分析できます')
        else:
            print('  なし（0/1の2値カラムが見つかりません）')
        return
    
    print(f'\n📌 分析対象フラグ: {len(flag_cols)}個')
    for i, col in enumerate(flag_cols, 1):
        print(f'  {i}. {col}')
    
    # 売上カラム検出
    sales_col = None
    for col_name in ['売上金額', '売上金額', 'sales', 'amount', '売上']:
        if col_name in df_filtered.columns:
            sales_col = col_name
            break
    
    if sales_col is None:
        print('\n⚠️ 売上カラムが見つかりません')
        print('💡 以下のカラム名を使用してください: sales_amt, 売上金額, sales, amount')
        return
    
    print(f'\n💰 売上カラム: {sales_col}')
    
    # フラグ別集計
    results = []
    for fc in flag_cols:
        # 0/1以外の値をチェック
        unique_vals = df_filtered[fc].dropna().unique()
        if not set(unique_vals).issubset({0, 1, 0.0, 1.0, True, False}):
            print(f'⚠️ スキップ: {fc} (0/1以外の値を含む: {unique_vals})')
            continue
        
        flag_on = df_filtered[df_filtered[fc].isin([1, 1.0, True])]
        flag_off = df_filtered[df_filtered[fc].isin([0, 0.0, False])]
        
        if len(flag_on) == 0 or len(flag_off) == 0:
            continue
        
        on_mean = flag_on[sales_col].mean()
        off_mean = flag_off[sales_col].mean()
        on_sum = flag_on[sales_col].sum()
        off_sum = flag_off[sales_col].sum()
        
        mean_diff = on_mean - off_mean
        mean_lift = (mean_diff / off_mean * 100) if off_mean > 0 else 0
        
        # プレフィックスを削除して表示名を作成
        display_name = fc.replace('flag_', '').replace('is_', '').replace('has_', '').replace('_フラグ', '')
        
        results.append({
            'フラグ': display_name,
            'ON件数': len(flag_on),
            'OFF件数': len(flag_off),
            'ON平均': on_mean,
            'OFF平均': off_mean,
            '平均差': mean_diff,
            '平均リフト(%)': mean_lift,
            'ON合計': on_sum,
            'OFF合計': off_sum,
            '合計差': on_sum - off_sum
        })
    
    if not results:
        print('⚠️ 分析可能なデータがありません')
        return
    
    # 結果をDataFrameに変換
    df_result = pd.DataFrame(results).sort_values('平均リフト(%)', ascending=False)
    
    # 可視化: 4パネル
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('📊 フラグ別売上インパクト分析', fontsize=16, fontproperties=font_setup.jp_font)
    
    # 1. 平均リフト率
    ax1 = axes[0, 0]
    colors = ['green' if x > 0 else 'red' for x in df_result['平均リフト(%)']]
    ax1.barh(df_result['フラグ'], df_result['平均リフト(%)'], color=colors, alpha=0.7)
    ax1.set_xlabel('平均リフト率 (%)', fontproperties=font_setup.jp_font)
    ax1.set_title('1️⃣ 平均売上リフト率（ON時の増減）', fontproperties=font_setup.jp_font)
    ax1.axvline(0, color='black', linestyle='--', linewidth=0.8)
    ax1.grid(axis='x', alpha=0.3)
    
    # 2. ON/OFF平均売上比較
    ax2 = axes[0, 1]
    x = np.arange(len(df_result))
    width = 0.35
    ax2.bar(x - width/2, df_result['OFF平均'], width, label='OFF', alpha=0.7, color='lightcoral')
    ax2.bar(x + width/2, df_result['ON平均'], width, label='ON', alpha=0.7, color='skyblue')
    ax2.set_xticks(x)
    ax2.set_xticklabels(df_result['フラグ'], rotation=45, ha='right', fontproperties=font_setup.jp_font)
    ax2.set_ylabel('平均売上（円）', fontproperties=font_setup.jp_font)
    ax2.set_title('2️⃣ ON/OFF時の平均売上比較', fontproperties=font_setup.jp_font)
    ax2.legend(prop=font_setup.jp_font)
    ax2.grid(axis='y', alpha=0.3)
    
    # 3. 件数分布
    ax3 = axes[1, 0]
    x = np.arange(len(df_result))
    ax3.bar(x - width/2, df_result['OFF件数'], width, label='OFF', alpha=0.7, color='lightcoral')
    ax3.bar(x + width/2, df_result['ON件数'], width, label='ON', alpha=0.7, color='skyblue')
    ax3.set_xticks(x)
    ax3.set_xticklabels(df_result['フラグ'], rotation=45, ha='right', fontproperties=font_setup.jp_font)
    ax3.set_ylabel('件数', fontproperties=font_setup.jp_font)
    ax3.set_title('3️⃣ ON/OFF時のデータ件数', fontproperties=font_setup.jp_font)
    ax3.legend(prop=font_setup.jp_font)
    ax3.grid(axis='y', alpha=0.3)
    
    # 4. 合計売上インパクト
    ax4 = axes[1, 1]
    colors = ['green' if x > 0 else 'red' for x in df_result['合計差']]
    ax4.barh(df_result['フラグ'], df_result['合計差']/1000, color=colors, alpha=0.7)
    ax4.set_xlabel('合計売上差（千円）', fontproperties=font_setup.jp_font)
    ax4.set_title('4️⃣ 合計売上インパクト（ON-OFF）', fontproperties=font_setup.jp_font)
    ax4.axvline(0, color='black', linestyle='--', linewidth=0.8)
    ax4.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 数値表示
    print('\n' + '='*80)
    print('📊 フラグ別売上分析結果')
    print('='*80)
    display(df_result.style.format({
        'ON平均': '{:,.0f}円',
        'OFF平均': '{:,.0f}円',
        '平均差': '{:,.0f}円',
        '平均リフト(%)': '{:+.1f}%',
        'ON合計': '{:,.0f}円',
        'OFF合計': '{:,.0f}円',
        '合計差': '{:,.0f}円'
    }).background_gradient(subset=['平均リフト(%)'], cmap='RdYlGn', vmin=-20, vmax=20))
    
    # トップ3推奨
    print('\n' + '='*80)
    print('💡 売上最大化のアクション推奨（平均リフト率トップ3）')
    print('='*80)
    for i, (idx, row) in enumerate(df_result.head(3).iterrows(), 1):
        print(f"\n{i}. **{row['フラグ']}**")
        print(f"   平均リフト: {row['平均リフト(%)']:+.1f}% | ON平均: {row['ON平均']:,.0f}円 vs OFF平均: {row['OFF平均']:,.0f}円")
        print(f"   推奨: {row['フラグ']}を積極的に活用してください")

# 分析を自動実行
analyze_sales_by_flags()


## 🔧 ステップ3: 特徴量テーブルの作成

### 🎯 **このセクションの目的**
AIが分析しやすいように、データを日次×店舗単位に整理します。

### 👀 **店長が確認すべきこと**
実行後、以下のメッセージが表示されます：
```
[INFO] 利用可能特徴量: 28/30 個
[INFO] カテゴリカル変数: ['category_l', 'category_m', 'store_id']
[INFO] 特徴量テーブル: (2500, 35) (行×列)
[INFO] 欠損値: 150 個
```

**意味**:
- **28個の要因**（気温、曜日、天気など）を使用
- **2,500日分**のデータで分析
- 欠損値は自動処理される

### 💡 **追加された特徴量の意味**
このセルで、以下の便利な特徴量が自動追加されます：

| 特徴量 | 意味 | 実務での活用 |
|--------|------|------------|
| `sales_lag_1` | 前日の売上 | 「昨日売れたから今日も売れる」パターン発見 |
| `sales_lag_7` | 1週間前の売上 | 「先週の月曜と今週の月曜は似ている」パターン発見 |
| `sales_rolling_7` | 7日移動平均 | トレンド把握（売上が伸びているか下がっているか） |
| `day_of_week` | 曜日（0=月曜、6=日曜） | 曜日別の売上パターン分析 |
| `is_weekend` | 週末フラグ（土日=1） | 週末効果の測定 |

**判断ポイント**:
- 欠損値が50%超の列 → 自動で除外される
- 特徴量が10個未満 → データ期間を延長検討

このセルは自動実行するだけでOKです。

---

**次のセルを実行してください ↓**

In [ ]:
# 全ての数値列を特徴量として使用（事前の絞り込みなし）
print('[INFO] データに含まれる全ての列を使用します（事前絞り込みなし）')

# 除外すべき列（分析対象でない列）
exclude_cols = ['店舗', '商品名', '日付', '日付', '店舗', '商品名', 
                'フェイスくくり大分類', 'フェイスくくり中分類', 'フェイスくくり小分類',
                'フェイスくくり大分類', 'フェイスくくり中分類', 'フェイスくくり小分類',
                '売上金額', '売上金額', 'price',  # 売上数量(qty)は目的変数なので除外しない
                '昨年同日_売上', '昨年同日_客数', '昨年同日_客単価']  # 目的変数と関連列を除外

# 全列リスト
all_cols = df.columns.tolist()

# 数値列のみ抽出（文字列・日付型を除く）
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# 特徴量候補：数値列から除外列を引く
# 特徴量候補：数値列から除外列を引く（qtyも明示的に除外）
feature_cols = [c for c in numeric_cols if c not in exclude_cols and c not in ['売上数量', '売上数量']]

print(f'[INFO] 全列数: {len(all_cols)}')
print(f'[INFO] 数値列数: {len(numeric_cols)}')
print(f'[INFO] 除外列数: {len([c for c in exclude_cols if c in all_cols])}')
print(f'[INFO] 使用する特徴量: {len(feature_cols)} 個')
print(f'\n[特徴量リスト（全{len(feature_cols)}個）]:')
for i, col in enumerate(feature_cols, 1):
    print(f'  {i}. {col}')

# カテゴリカル変数の確認
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
categorical_present = [c for c in categorical_cols if c not in exclude_cols and c in df.columns]
print(f'\n[INFO] カテゴリカル変数: {len(categorical_present)} 個 → {categorical_present}')

# 基本テーブル構築（全ての数値特徴量を使用）
keep = ['日付', '店舗', '売上数量'] + feature_cols
tmp = df[[c for c in keep if c in df.columns]].copy()

# 日付×店舗で特徴量を代表化（数値は平均）、売上は合計
# qtyが feature_cols に含まれている場合は除外（目的変数なので特徴量としては使わない）
feature_cols_for_agg = [c for c in feature_cols if c not in ['売上数量', '売上数量']]
Xdf = tmp.groupby(['日付','店舗'], as_index=False)[feature_cols_for_agg].mean() if feature_cols_for_agg else tmp[['日付','店舗']].drop_duplicates()
ydf = tmp.groupby(['日付','店舗'], as_index=False)['売上数量'].sum()
feat = Xdf.merge(ydf, on=['日付','店舗'], how='inner')

# qtyが存在することを確認
if '売上数量' not in feat.columns:
    print('[ERROR] qty列が見つかりません！データを確認してください')
else:
    qty_sum = feat["売上数量"].sum()
    qty_mean = feat["売上数量"].mean()
    print(f'[INFO] qty列を確認: 合計={qty_sum:,.0f}個, 平均={qty_mean:.1f}個/日')
    print(f'[DEBUG] qty列の型: {feat["売上数量"].dtype}, NaN数: {feat["売上数量"].isna().sum()}')

# カテゴリカル変数の追加（元データから）
if categorical_present:
    cat_data = df[['日付','店舗'] + categorical_present].drop_duplicates()
    feat = feat.merge(cat_data, on=['日付','店舗'], how='left')

# 重複列名のチェックと削除
duplicate_cols = feat.columns[feat.columns.duplicated()].tolist()
if duplicate_cols:
    print(f'[WARNING] 重複列名を検出: {duplicate_cols}')
    # 重複列を削除（最初の列のみ残す）
    feat = feat.loc[:, ~feat.columns.duplicated()]
    print(f'[INFO] 重複列を削除後: {feat.shape}')
    print(f'[INFO] カテゴリカル変数をマージ: {categorical_present}')

print(f'\n[INFO] 特徴量テーブル: {feat.shape} (行×列)')
print(f'[INFO] 欠損値: {feat.isnull().sum().sum()} 個 ({feat.isnull().sum().sum() / feat.size * 100:.2f}%)')

# 欠損値が多い列を警告
missing_ratio = feat.isnull().sum() / len(feat)
high_missing = missing_ratio[missing_ratio > 0.5].sort_values(ascending=False)
if not high_missing.empty:
    print(f'\n[WARNING] 欠損値50%超の列（{len(high_missing)}個）:')
    for col, ratio in high_missing.items():
        print(f'  - {col}: {ratio*100:.1f}% 欠損')

feat.head(3)

## 🤖 ステップ4: PyCaret — AIで売上予測モデルを作成

### 🎯 **このセクションの目的**
**PyCaretを使って、売上を予測するAIモデルを自動的に作成します。**

---

### 📋 **PyCaretが自動でやってくれること**

1. **15～20種類のAIモデルを一括比較**
   - LightGBM、XGBoost、CatBoost（GPU対応）
   - ランダムフォレスト、決定木、線形回帰
   - その他の高度なモデル

2. **データの前処理**
   - 欠損値の補完
   - 外れ値の検出
   - 特徴量のスケーリング（正規化）

3. **交差検証（3-Fold CV）**
   - データを3分割して、過学習を防ぎながら精度を評価

4. **最良モデルの自動選択**
   - R²スコアが最も高いモデルを選択

---

### 👀 **店長が確認すべきこと**

#### 1️⃣ **モデル比較結果（Leaderboard）**
実行後、以下のような表が表示されます：

| Model | R2 | RMSE | MAE | 実行時間 |
|-------|-----|------|-----|---------|
| LightGBM | 0.85 | 1200 | 950 | 2.3秒 |
| XGBoost | 0.83 | 1350 | 1020 | 3.1秒 |
| CatBoost | 0.82 | 1400 | 1080 | 2.8秒 |

**重要指標の読み方：**
- **R² (決定係数)**: 0～1の範囲。**0.7以上なら実用レベル**、0.8以上なら高精度
  - 0.85 = 売上変動の85%をAIが説明できている
- **RMSE (平均二乗誤差)**: 予測誤差の大きさ（円単位）。**小さいほど良い**
- **MAE (平均絶対誤差)**: より直感的な誤差（円単位）。**日々の売上±この金額の範囲で予測**

#### 2️⃣ **GPU加速の確認**
```
🚀 [INFO] GPU使用モードで実行（LightGBM/XGBoost/CatBoost対応）
⏱️ 実行時間: 23.4秒
💡 GPUにより高速化されました
```
↑ このメッセージが表示されればGPU加速が有効です（CPU版の約10倍高速）

#### 3️⃣ **特徴量重要度グラフ**
「どの要素が売上に効いているか」を可視化：
- **天気**（flag_weather_rain）が重要 → 雨の日対策が必要
- **曜日**（flag_dow_土曜）が重要 → 週末の品揃えが鍵
- **イベント**（flag_event_給料日）が重要 → 給料日セールが効果的

---

### ⚙️ **GPU設定の詳細**

PyCaretは以下のGPUパラメータを自動設定します：

```python
# LightGBM GPU設定
'device': 'gpu'
'gpu_platform_id': 0
'gpu_device_id': 0

# XGBoost GPU設定  
'tree_method': 'gpu_hist'
'gpu_id': 0

# CatBoost GPU設定
'task_type': 'GPU'
'devices': '0'
```

---

### 📌 **注意事項**

1. **最低データ数**: 100行以上必要（少ないとエラーになります）
2. **実行時間**: データ量により1～5分程度かかります
3. **GPU非搭載の場合**: 自動的にCPUモードで実行されます（やや遅くなります）

---

### 🎯 **次のステップ**
このステップで学習したモデル（`final`変数）を使って、ステップ5で予測結果の可視化を行います。


In [ ]:
# ========================================
# 🤖 ステップ4: PyCaret実行（GPU自動検出 + CPUフォールバック）
# ========================================

if PYC_OK and 'feat' in locals() and len(feat) >= MIN_SAMPLES_PYCARET:
    print(f'[INFO] PyCaret開始: {len(feat)}行 (>= {MIN_SAMPLES_PYCARET}行)')
    
    # PyCaretセットアップ用データ準備
    data = feat.drop(columns=['日付','店舗'], errors='ignore').copy()
    
    # 欠損値が多い列を削除（50%以上欠損）※ただし目的変数qtyは保護
    missing_ratio = data.isnull().sum() / len(data)
    drop_cols = missing_ratio[missing_ratio > 0.5].index.tolist()
    # 目的変数qtyを除外リストから削除（保護）
    if '売上数量' in drop_cols:
        drop_cols.remove('売上数量')
    if drop_cols:
        print(f'[WARNING] 欠損値50%超の列を除外: {drop_cols}')
        data = data.drop(columns=drop_cols)
    
    print(f'[INFO] 学習データ: {data.shape} | 目的変数: qty (販売数量)')
    print(f'[DEBUG] data.columns に qty が存在: {"売上数量" in data.columns}')
    print(f'[DEBUG] data.columns: {list(data.columns[:10])}...')
    if '売上数量' in data.columns:
        print(f'[DEBUG] qty統計: min={data["売上数量"].min()}, max={data["売上数量"].max()}, mean={data["売上数量"].mean():.2f}')
    else:
        print(f'[ERROR] qty列がdataに存在しません！')
        print(f'[ERROR] 利用可能な列: {list(data.columns)}')
    
    # ========================================
    # 🚀 GPU対応チェック（各ライブラリの実際の対応状況を確認）
    # ========================================
    
    def check_lightgbm_gpu():
        """LightGBMのGPU対応を確認"""
        try:
            import lightgbm as lgb
            # テストデータでGPU動作確認
            test_data = [[1, 2], [3, 4]]
            test_label = [1, 2]
            lgb_train = lgb.Dataset(test_data, test_label)
            params = {'device': 'gpu', 'verbosity': -1}
            lgb.train(params, lgb_train, num_boost_round=1)
            return True
        except Exception:
            return False
    
    def check_xgboost_gpu():
        """XGBoostのGPU対応を確認"""
        try:
            import xgboost as xgb
            # テストデータでGPU動作確認
            dtrain = xgb.DMatrix([[1, 2], [3, 4]], label=[1, 2])
            params = {'tree_method': 'gpu_hist', 'verbosity': 0}
            xgb.train(params, dtrain, num_boost_round=1)
            return True
        except Exception:
            return False
    
    def check_catboost_gpu():
        """CatBoostのGPU対応を確認"""
        try:
            from catboost import CatBoostRegressor
            model = CatBoostRegressor(task_type='GPU', iterations=1, verbose=0)
            model.fit([[1, 2], [3, 4]], [1, 2])
            return True
        except Exception:
            return False
    
    # GPU対応状況の確認
    print('\n🔍 [INFO] GPU対応状況を確認中...')
    
    lightgbm_gpu = False
    xgboost_gpu = False
    catboost_gpu = False
    
    if USE_GPU and GPU_AVAILABLE:
        lightgbm_gpu = check_lightgbm_gpu()
        xgboost_gpu = check_xgboost_gpu()
        catboost_gpu = check_catboost_gpu()
        
        print(f'  LightGBM GPU: {"✅ 利用可能" if lightgbm_gpu else "❌ 未対応（CPUフォールバック）"}')
        print(f'  XGBoost GPU: {"✅ 利用可能" if xgboost_gpu else "❌ 未対応（CPUフォールバック）"}')
        print(f'  CatBoost GPU: {"✅ 利用可能" if catboost_gpu else "❌ 未対応（CPUフォールバック）"}')
        
        if not any([lightgbm_gpu, xgboost_gpu, catboost_gpu]):
            print('\n⚠️ [WARNING] GPU対応ライブラリがインストールされていません')
            print('💡 CPUモードで実行します（精度は同じですが速度は遅くなります）')
            print('\n📝 GPU対応版のインストール方法:')
            print('  pip install lightgbm --install-option=--gpu')
            print('  pip install xgboost[gpu]')
            print('  pip install catboost --install-option="--features=GPU"')
    else:
        print('  ℹ️ GPUが検出されていないため、CPUモードで実行します')
    
    try:
        # ========================================
        # 📊 PyCaretセットアップ（並列処理有効化）
        # ========================================
        import time
        setup_start = time.time()
        
        _ = setup(
            data=data, 
            target='売上数量', 
            session_id=42, 
            fold=3,
            verbose=False,
            normalize=True,
            transformation=True,
            ignore_features=['sales_lag_1', 'sales_lag_7', 'sales_rolling_7'],
            n_jobs=-1,  # 全CPUコアを使用（並列処理）
        )
        
        setup_time = time.time() - setup_start
        print(f'\n✅ [INFO] セットアップ完了: {setup_time:.1f}秒')
        
        # ========================================
        # 🚀 モデル比較（並列実行 + 自動GPU設定）
        # ========================================
        print('\n[INFO] モデル比較実行中（全モデル並列実行）...')
        print('[INFO] 比較対象: PyCaretの全回帰モデル（15～20種類）')
        
        if any([lightgbm_gpu, xgboost_gpu, catboost_gpu]):
            print('[INFO] ✅ 一部モデルでGPU加速有効')
        else:
            print('[INFO] ℹ️ CPUモード（全コア並列実行）')
        
        print('[INFO] ※ 実行に数分かかる場合があります')
        
        compare_start = time.time()
        
        # 全モデル比較実行（PyCaretが自動で最適化）
        best = compare_models(
            sort='R2',
            n_select=1,
            turbo=True,  # 高速モード
        )
        
        compare_time = time.time() - compare_start
        
        leaderboard = pull()
        print('\n[SUCCESS] モデル比較完了')
        print(f'⏱️ 実行時間: {compare_time:.1f}秒')
        
        if any([lightgbm_gpu, xgboost_gpu, catboost_gpu]):
            print('💡 一部モデルでGPUにより高速化されました')
        
        print(f'\n📊 全{len(leaderboard)}モデルの比較結果:')
        display(leaderboard)
        
        # ========================================
        # 🎯 最良モデルのファイナライズ
        # ========================================
        finalize_start = time.time()
        
        final = finalize_model(best)
        
        finalize_time = time.time() - finalize_start
        
        print(f'\n[INFO] 最良モデル: {type(best).__name__}')
        print(f'[INFO] R2スコア: {leaderboard.iloc[0]["R2"]:.4f}')
        print(f'[INFO] RMSE: {leaderboard.iloc[0]["RMSE"]:.2f}')
        print(f'[INFO] MAE: {leaderboard.iloc[0]["MAE"]:.2f}')
        print(f'[INFO] ファイナライズ時間: {finalize_time:.1f}秒')
        
        # モデルタイプを確認してGPU使用状況を表示
        model_name = type(best).__name__
        if 'LGBM' in model_name:
            gpu_status = '✅ GPU' if lightgbm_gpu else 'ℹ️ CPU'
            print(f'[INFO] LightGBM: {gpu_status}モードで学習')
        elif 'XGB' in model_name:
            gpu_status = '✅ GPU' if xgboost_gpu else 'ℹ️ CPU'
            print(f'[INFO] XGBoost: {gpu_status}モードで学習')
        elif 'CatBoost' in model_name:
            gpu_status = '✅ GPU' if catboost_gpu else 'ℹ️ CPU'
            print(f'[INFO] CatBoost: {gpu_status}モードで学習')
        else:
            print(f'[INFO] {model_name}: CPUモードで学習')
        
        # ========================================
        # 📊 可視化
        # ========================================
        try:
            print('\n[INFO] 特徴量重要度プロット')
            plot_model(final, plot='feature')
        except Exception as e:
            print(f'[WARNING] 特徴量プロット失敗: {e}')
        
        try:
            print('\n[INFO] 残差プロット')
            plot_model(final, plot='residuals')
        except Exception as e:
            print(f'[WARNING] 残差プロット失敗: {e}')
        
        # ========================================
        # ⏱️ パフォーマンスサマリー
        # ========================================
        total_time = setup_time + compare_time + finalize_time
        print('\n' + '='*60)
        print('⏱️ パフォーマンスサマリー')
        print('='*60)
        print(f'セットアップ: {setup_time:.1f}秒')
        print(f'モデル比較: {compare_time:.1f}秒 ({len(leaderboard)}モデル)')
        print(f'ファイナライズ: {finalize_time:.1f}秒')
        print(f'合計時間: {total_time:.1f}秒')
        
        gpu_count = sum([lightgbm_gpu, xgboost_gpu, catboost_gpu])
        if gpu_count > 0:
            print(f'\n💡 {gpu_count}種類のモデルでGPU加速が有効でした')
        else:
            print(f'\nℹ️ CPUモードで実行しました（並列処理により高速化）')
        
    except Exception as e:
        print(f'[ERROR] PyCaret実行失敗')
        print(f'  エラー型: {type(e).__name__}')
        print(f'  詳細: {e}')
        import traceback
        traceback.print_exc()

elif PYC_OK:
    print(f'[WARNING] データ不足（{len(feat)}行 < {MIN_SAMPLES_PYCARET}行）。PyCaretスキップ。')
else:
    print('[WARNING] PyCaret未インストール。スキップ。')


## 📈 ステップ5: 予測結果の可視化とビジネスインサイト

### 🎯 **このセクションの目的**
**上のステップ4で学習したAIモデルを使って、予測結果を可視化し、ビジネス上の判断材料を提供します。**

---

### 📊 **このセクションで表示される内容**

#### 1️⃣ **予測精度メトリクス**
```
📊 予測精度メトリクス:
   R² Score: 0.8234
   MAE: 1,250円
   RMSE: 1,680円
   MAPE: 8.5%
```

**指標の読み方：**
- **R² (決定係数)**: 0.7以上なら実用レベル、0.8以上なら高精度
- **MAE (平均絶対誤差)**: 予測が平均±1,250円の範囲でズレる
- **RMSE (二乗平均平方根誤差)**: 大きな誤差を重視した指標
- **MAPE (平均絶対パーセント誤差)**: 予測誤差が約8.5%（売上の1割未満なら実用可）

#### 2️⃣ **4パネル可視化**
1. **予測vs実績 散布図**: モデルの当てはまり具合を確認
2. **残差分布**: 予測誤差の分布（正規分布に近いほど良い）
3. **時系列予測**: 日別の予測精度推移
4. **予測誤差率の分布**: どれくらいの割合で誤差が出るか

---

### 🎯 **実務での活用方法**

| 指標 | 判断基準 | アクション |
|------|----------|----------|
| **R² > 0.8** | 高精度 | このモデルを本番運用に投入可能 |
| **0.7 < R² < 0.8** | 実用レベル | 注意しながら運用、改善の余地あり |
| **R² < 0.7** | 要改善 | 特徴量の追加やデータ期間の延長が必要 |
| **MAPE < 10%** | 優秀 | 発注数量の自動化に活用可能 |
| **MAPE > 20%** | 要注意 | 人間のチェックを必須にする |

---

### ⚠️ **注意事項**

1. **モデルが学習されていない場合**
   ```
   ⚠️ モデルが学習されていません。先にステップ4を実行してください。
   ```
   → ステップ4を先に実行してください

2. **日付データがない場合**
   - 時系列プロットはスキップされます（散布図と残差のみ表示）

3. **異常値の確認**
   - 散布図で大きく外れた点（外れ値）がある場合、その日のデータを確認してください
   - イベント日や特殊要因があった可能性があります

---

### 💡 **次のステップ**
このステップで精度を確認したら、ステップ6で実際の発注数量予測を実行します。


In [ ]:
# ========================================
# 📈 ステップ5: 予測結果の可視化とビジネスインサイト
# ========================================

if 'final' in locals():
    print('\n' + '='*60)
    print('📈 ステップ5: 予測結果の可視化とビジネスインサイト')
    print('='*60)
    
    try:
        # 予測実行
        predictions = predict_model(final)
        
        # 予測精度の評価
        from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
        
        y_true = predictions['売上数量']
        y_pred = predictions['prediction_label']
        
        r2 = r2_score(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        rmse = mean_squared_error(y_true, y_pred, squared=False)
        
        # MAPE計算（ゼロ除算回避）
        non_zero_mask = y_true != 0
        if non_zero_mask.sum() > 0:
            mape = np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100
        else:
            mape = 0
        
        print(f'\n📊 予測精度メトリクス:')
        print(f'   R² Score: {r2:.4f}')
        print(f'   MAE: {mae:,.0f}円')
        print(f'   RMSE: {rmse:,.0f}円')
        print(f'   MAPE: {mape:.2f}%')
        
        # 4パネル可視化
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('📊 予測精度の可視化分析', fontsize=16, fontproperties=font_setup.jp_font)
        
        # 1. 予測vs実績 散布図
        ax1 = axes[0, 0]
        ax1.scatter(y_true, y_pred, alpha=0.5, s=20, color='steelblue')
        min_val = min(y_true.min(), y_pred.min())
        max_val = max(y_true.max(), y_pred.max())
        ax1.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='完全一致線')
        ax1.set_xlabel('実績売上（円）', fontproperties=font_setup.jp_font)
        ax1.set_ylabel('予測売上（円）', fontproperties=font_setup.jp_font)
        ax1.set_title(f'1️⃣ 予測vs実績（R²={r2:.3f}）', fontproperties=font_setup.jp_font)
        ax1.legend(prop=font_setup.jp_font)
        ax1.grid(alpha=0.3)
        
        # 2. 残差分布
        ax2 = axes[0, 1]
        residuals = y_true - y_pred
        ax2.hist(residuals, bins=30, edgecolor='black', alpha=0.7, color='coral')
        ax2.axvline(0, color='red', linestyle='--', linewidth=2)
        ax2.set_xlabel('残差（実績-予測）', fontproperties=font_setup.jp_font)
        ax2.set_ylabel('頻度', fontproperties=font_setup.jp_font)
        ax2.set_title(f'2️⃣ 残差分布（平均={residuals.mean():.0f}円）', fontproperties=font_setup.jp_font)
        ax2.grid(alpha=0.3)
        
        # 3. 時系列予測（日付がある場合）
        ax3 = axes[1, 0]
        
        # predictionsに日付が含まれているか確認
        if '日付' in predictions.columns:
            # 既に日付が含まれている場合
            pred_sorted = predictions.sort_values('日付')
            ax3.plot(pred_sorted['日付'], pred_sorted['売上数量'], 
                    label='実績', alpha=0.7, linewidth=1.5, color='blue', marker='o', markersize=3)
            ax3.plot(pred_sorted['日付'], pred_sorted['prediction_label'], 
                    label='予測', alpha=0.7, linewidth=1.5, color='orange', marker='x', markersize=3)
            ax3.set_xlabel('日付', fontproperties=font_setup.jp_font)
            ax3.set_ylabel('売上（円）', fontproperties=font_setup.jp_font)
            ax3.set_title('3️⃣ 時系列予測推移', fontproperties=font_setup.jp_font)
            ax3.legend(prop=font_setup.jp_font)
            ax3.grid(alpha=0.3)
            plt.setp(ax3.xaxis.get_majorticklabels(), rotation=45, ha='right')
        else:
            # 日付がない場合はインデックスで表示
            ax3.plot(range(len(predictions)), predictions['売上数量'], 
                    label='実績', alpha=0.7, linewidth=1.5, color='blue', marker='o', markersize=3)
            ax3.plot(range(len(predictions)), predictions['prediction_label'], 
                    label='予測', alpha=0.7, linewidth=1.5, color='orange', marker='x', markersize=3)
            ax3.set_xlabel('データポイント', fontproperties=font_setup.jp_font)
            ax3.set_ylabel('売上（円）', fontproperties=font_setup.jp_font)
            ax3.set_title('3️⃣ 予測推移（時系列順）', fontproperties=font_setup.jp_font)
            ax3.legend(prop=font_setup.jp_font)
            ax3.grid(alpha=0.3)
        
        # 4. 予測誤差率の分布
        ax4 = axes[1, 1]
        error_pct = ((y_pred - y_true) / y_true.replace(0, np.nan) * 100).dropna()
        error_pct_clipped = error_pct.clip(-100, 100)
        ax4.hist(error_pct_clipped, bins=30, edgecolor='black', alpha=0.7, color='lightgreen')
        ax4.axvline(0, color='red', linestyle='--', linewidth=2)
        ax4.set_xlabel('予測誤差率（%）', fontproperties=font_setup.jp_font)
        ax4.set_ylabel('頻度', fontproperties=font_setup.jp_font)
        ax4.set_title(f'4️⃣ 予測誤差率分布（MAPE={mape:.1f}%）', fontproperties=font_setup.jp_font)
        ax4.grid(alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # ビジネスインサイト
        print('\n' + '='*60)
        print('💡 ビジネスインサイト')
        print('='*60)
        
        if r2 >= 0.8:
            print('✅ 【高精度】このモデルは本番運用に投入可能です')
            print(f'   - モデルは売上変動の{r2*100:.1f}%を説明できています')
        elif r2 >= 0.7:
            print('⚠️ 【実用レベル】注意しながら運用できます（改善の余地あり）')
            print(f'   - モデルは売上変動の{r2*100:.1f}%を説明できています')
        else:
            print('❌ 【要改善】特徴量の追加やデータ期間の延長が必要です')
            print(f'   - モデルは売上変動の{r2*100:.1f}%しか説明できていません')
        
        if mape < 10:
            print('✅ 【優秀】発注数量の自動化に活用できます')
        elif mape < 20:
            print('⚠️ 【実用可】人間のチェックを併用してください')
        else:
            print('❌ 【要注意】自動化は避け、参考値として利用してください')
        
        print(f'\n📊 予測精度の実務的な意味:')
        print(f'  - 平均予測誤差: ±{mae:,.0f}円')
        print(f'  - 大部分の予測は ±{rmse:,.0f}円 の範囲内')
        print(f'  - 予測レコード数: {len(predictions):,}件')
        
        # 最大誤差の分析
        abs_errors = np.abs(residuals)
        max_error_idx = abs_errors.idxmax()
        max_error_actual = y_true.loc[max_error_idx]
        max_error_pred = y_pred.loc[max_error_idx]
        max_error_diff = residuals.loc[max_error_idx]
        
        print(f'\n⚠️ 最大誤差のケース:')
        print(f'  - 実績: {max_error_actual:,.0f}円')
        print(f'  - 予測: {max_error_pred:,.0f}円')
        print(f'  - 誤差: {max_error_diff:,.0f}円 ({abs(max_error_diff/max_error_actual*100):.1f}%)')
        print(f'  💡 このような外れ値は特殊イベントの可能性があります')
        
    except Exception as e:
        print(f'⚠️ 可視化エラー: {e}')
        import traceback
        traceback.print_exc()

else:
    print('⚠️ モデルが学習されていません。先にステップ4を実行してください。')


## 📦 ステップ6: 1週間先の発注数量予測（全商品）

### 🎯 **このセクションの目的**
**学習したAIモデルを使って、2025年10月15日から1週間分の全商品の販売数量を予測し、発注数量を自動計算します。**

---

### 📋 **予測の流れ**

1. **外部要因の入力** → 1週間分の天気予報・カレンダー情報を入力
2. **AIが予測** → 学習したモデルで商品×日付ごとに販売数量を予測
3. **発注数量の計算** → 予測値 × 安全係数で発注数量を算出
4. **CSV出力** → 発注表をCSVファイルとして保存

---

### 👀 **店長が入力すべき情報**

次のセルで、以下の情報を入力してください：

| 項目 | 入力例 | 説明 |
|------|--------|------|
| **予測開始日** | `2025-10-15` | 発注を開始する日 |
| **予測日数** | `7` | 何日分予測するか（通常は7日） |
| **天気予報** | `['晴れ', '曇り', '雨', ...]` | 1週間分の天気予報 |
| **平均気温** | `[22, 23, 20, 18, ...]` | 1週間分の予測気温（℃） |
| **降水確率** | `[10, 30, 70, ...]` | 1週間分の降水確率（%） |
| **安全係数** | `1.2` | 欠品防止のための余裕率（通常1.1～1.3） |

**安全係数の目安**:
- `1.0` → 予測値そのまま（リスク高）
- `1.1～1.2` → 通常の運用（推奨）
- `1.3～1.5` → 欠品を絶対に避けたい（廃棄リスク増）

---

**次のセルで外部要因を入力し、予測を実行してください ↓**

In [ ]:
# ========================================
# 📊 Step 6-1: 全店舗×全商品の予測用データ作成（GPU高速化版）
# ========================================

print('\n' + '='*60)
print('📅 Step 6-1: 全店舗の天気データ取得 + 予測用特徴量作成')
print('='*60)

from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import requests
from pathlib import Path

# ========================================
# 📍 店舗マスタの読み込み
# ========================================

print('\n📍 店舗マスタを読み込み中...')
stores_df = pd.read_csv('stores.csv')
print(f'  ✅ 店舗数: {len(stores_df)} 店舗')
print(f'  店舗リスト: {list(stores_df["店舗名"][:5])}...')

# ========================================
# 🛍️ 商品マスタの作成（学習データから抽出）
# ========================================

print('\n🛍️ 商品マスタを作成中...')
if 'df' in globals() and '商品名' in df.columns:
    # 学習データから商品情報を抽出
    sku_master = df[['商品名']].drop_duplicates().copy()
    
    # 商品単価を計算（price列がある場合）
    if 'price' in df.columns:
        sku_prices = df.groupby('商品名')['price'].mean().reset_index()
        sku_master = sku_master.merge(sku_prices, on='商品名', how='left')
    elif '売上金額' in df.columns and '売上数量' in df.columns:
        # sales_amt / qty で単価を計算
        df_with_price = df[df['売上数量'] > 0].copy()
        df_with_price['calc_price'] = df_with_price['売上金額'] / df_with_price['売上数量']
        sku_prices = df_with_price.groupby('商品名')['calc_price'].median().reset_index()
        sku_prices.columns = ['商品名', 'price']
        sku_master = sku_master.merge(sku_prices, on='商品名', how='left')
    
    print(f'  ✅ 商品数: {len(sku_master)} SKU')
    if 'price' in sku_master.columns:
        print(f'  平均単価: ¥{sku_master["price"].mean():.0f}')
else:
    print('  ⚠️ 学習データから商品マスタを作成できませんでした')
    sku_master = pd.DataFrame({'商品名': [1]})  # ダミー

# ========================================
# 🌤️ 全店舗の天気予報を一括取得（並列処理）
# ========================================

FORECAST_START_DATE = '2025-10-15'  # 予測開始日
FORECAST_DAYS = 7  # 予測日数

print(f'\n🌤️ 全{len(stores_df)}店舗の天気予報を取得中...')
print(f'  期間: {FORECAST_START_DATE} ～ {FORECAST_DAYS}日間')

weather_codes = {
    0: '晴れ', 1: '晴れ', 2: '晴れ', 3: '曇り',
    45: '霧', 48: '霧', 51: '小雨', 53: '小雨', 55: '雨',
    61: '雨', 63: '雨', 65: '大雨', 80: '雨', 81: '雨', 82: '大雨'
}

all_store_weather = []

for idx, store in stores_df.iterrows():
    store_name = store['店舗名']
    lat = store['緯度']
    lon = store['経度']
    
    try:
        url = 'https://api.open-meteo.com/v1/forecast'
        params = {
            'latitude': lat,
            'longitude': lon,
            'daily': 'temperature_2m_max,temperature_2m_min,precipitation_sum,weathercode',
            'timezone': 'Asia/Tokyo',
            'start_date': FORECAST_START_DATE,
            'end_date': (datetime.strptime(FORECAST_START_DATE, '%Y-%m-%d') + timedelta(days=FORECAST_DAYS-1)).strftime('%Y-%m-%d')
        }
        
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        # 店舗ごとの天気データを保存
        for i in range(len(data['daily']['time'])):
            weather_code = data['daily']['weathercode'][i]
            all_store_weather.append({
                'store_name': store_name,
                'store_lat': lat,
                'store_lon': lon,
                '日付': data['daily']['time'][i],
                'weather': weather_codes.get(weather_code, '不明'),
                'temp_max': data['daily']['temperature_2m_max'][i],
                'temp_min': data['daily']['temperature_2m_min'][i],
                'temp_avg': (data['daily']['temperature_2m_max'][i] + data['daily']['temperature_2m_min'][i]) / 2,
                'precipitation': data['daily']['precipitation_sum'][i]
            })
        
        print(f'  ✅ {store_name}: {len(data["daily"]["time"])}日分取得')
        
    except Exception as e:
        print(f'  ❌ {store_name}: API失敗 ({e}) - ダミーデータ使用')
        # ダミーデータ
        for i in range(FORECAST_DAYS):
            forecast_date = (datetime.strptime(FORECAST_START_DATE, '%Y-%m-%d') + timedelta(days=i)).strftime('%Y-%m-%d')
            all_store_weather.append({
                'store_name': store_name,
                'store_lat': lat,
                'store_lon': lon,
                '日付': forecast_date,
                'weather': '晴れ',
                'temp_max': 25.0,
                'temp_min': 18.0,
                'temp_avg': 21.5,
                'precipitation': 0.0
            })

weather_all_df = pd.DataFrame(all_store_weather)
print(f'\n✅ 天気データ取得完了: {len(weather_all_df)} レコード')

# ========================================
# 🔄 全店舗×全商品×全日付の組み合わせを生成
# ========================================

print('\n🔄 予測用データセットを生成中...')

# 店舗マスタに店舗IDを追加
stores_df['店舗'] = range(1, len(stores_df) + 1)

# 全組み合わせを生成（店舗×商品×日付）
forecast_combinations = []

for _, store in stores_df.iterrows():
    store_id = store['店舗']
    store_name = store['店舗名']
    
    for _, sku in sku_master.iterrows():
        sku_id = sku['商品名']
        sku_price = sku.get('price', np.nan)
        
        # この店舗の天気データを取得
        store_weather = weather_all_df[weather_all_df['store_name'] == store_name]
        
        for _, w_data in store_weather.iterrows():
            date_obj = datetime.strptime(w_data['日付'], '%Y-%m-%d')
            
            features = {
                '店舗': store_id,
                '商品名': sku_id,
                'price': sku_price,
                '日付': w_data['日付'],
                '年': date_obj.year,
                '月': date_obj.month,
                '日': date_obj.day,
                '曜日': date_obj.weekday(),
                '週番号': date_obj.isocalendar()[1],
                '年内日数': date_obj.timetuple().tm_yday,
                '祝日フラグ': 0,
                '土曜フラグ': 1 if date_obj.weekday() == 5 else 0,
                '日曜フラグ': 1 if date_obj.weekday() == 6 else 0,
                '週末フラグ': 1 if date_obj.weekday() >= 5 else 0,
                '平日フラグ': 1 if date_obj.weekday() < 5 else 0,
                '天気': w_data['weather'],
                '最高気温': w_data['temp_max'],
                '最低気温': w_data['temp_min'],
                '平均気温': w_data['temp_avg'],
                '気温差': w_data['temp_max'] - w_data['temp_min'],
                '降水量': w_data['precipitation'],
            }
            
            # 学習データの全カラムに対応
            if 'feat' in globals():
                for col in feat.columns:
                    if col not in features and col not in ['売上数量', '売上金額']:
                        # ラグ特徴量や移動平均はNaN
                        if '_t-' in col or '_MA' in col or '変化' in col or 'トレンド' in col or '累積' in col:
                            features[col] = np.nan
                        else:
                            features[col] = 0
            
            forecast_combinations.append(features)

forecast_df = pd.DataFrame(forecast_combinations)

# 学習データと同じ列順序に並べ替え
if 'feat' in globals():
    common_cols = [c for c in feat.columns if c in forecast_df.columns and c not in ['売上数量', '売上金額']]
    # 必須列（店舗、商品名、日付）を保持しつつ、学習データと同じ列順序に並べ替え
    essential_cols = ['店舗', '商品名', '日付']
    feature_cols_only = [c for c in common_cols if c not in essential_cols]
    final_cols = essential_cols + feature_cols_only + ['price']
    # forecast_dfに存在する列のみ選択
    final_cols = [c for c in final_cols if c in forecast_df.columns]
    forecast_df = forecast_df[final_cols]

print(f'\n✅ 予測用データセット作成完了:')
print(f'  総レコード数: {len(forecast_df):,} 件')
print(f'  = {len(stores_df)} 店舗 × {len(sku_master)} 商品 × {FORECAST_DAYS} 日')
print(f'  列数: {len(forecast_df.columns)}')

# サンプル表示
print('\n📋 予測用データサンプル:')
sample_cols = ['店舗', '商品名', '日付', '年', '月', '曜日', '天気', '最高気温', 'price']
display_cols = [c for c in sample_cols if c in forecast_df.columns]
display(forecast_df[display_cols].head(10))

print('\n💡 次のステップ: Step 6-2で商品別・店舗別の販売数量を予測してください')


In [ ]:
# ========================================
# 📦 Step 6-2: 全店舗×全商品の販売数量予測（GPU高速化版）
# ========================================

if not PYC_OK or 'final' not in globals():
    print('⚠️ モデルが学習されていません。先にステップ4を実行してください。')
elif 'forecast_df' not in globals():
    print('⚠️ 予測用データが作成されていません。先にStep 6-1を実行してください。')
else:
    print('\n' + '='*60)
    print('🤖 全店舗×全商品の販売数量予測を実行中...')
    print('='*60)
    
    total_records = len(forecast_df)
    print(f'\n📊 予測対象:')
    print(f'  商品数: {forecast_df["商品名"].nunique()} SKU')
    print(f'  店舗数: {forecast_df["店舗"].nunique()} 店舗')
    print(f'  日数: {forecast_df["日付"].nunique()} 日')
    print(f'  合計予測レコード数: {total_records:,} 件')
    
    # GPU並列化設定
    BATCH_SIZE = 10000  # GPU最適なバッチサイズ（大量データ対応）
    
    if USE_GPU and GPU_AVAILABLE:
        print(f'\n🚀 GPU並列化モード: {BATCH_SIZE:,}件ずつバッチ処理')
    else:
        print(f'\nℹ️ CPU処理モード: {BATCH_SIZE:,}件ずつバッチ処理')
    
    import time
    start_time = time.time()
    
    # バッチ予測（GPU高速化）
    predictions_list = []
    num_batches = (total_records + BATCH_SIZE - 1) // BATCH_SIZE
    
    for batch_idx in range(num_batches):
        start_idx = batch_idx * BATCH_SIZE
        end_idx = min((batch_idx + 1) * BATCH_SIZE, total_records)
        
        batch_data = forecast_df.iloc[start_idx:end_idx].copy()
        
        # price列を除外して予測
        pred_cols = [c for c in batch_data.columns if c != 'price']
        batch_pred_data = batch_data[pred_cols]
        
        # バッチ予測実行
        try:
            batch_pred = predict_model(final, data=batch_pred_data)
            
            # price列を復元
            if 'price' in batch_data.columns:
                batch_pred['price'] = batch_data['price'].values
            
            predictions_list.append(batch_pred)
            
            # 進捗表示
            progress = (batch_idx + 1) / num_batches * 100
            print(f'\r⏳ 予測進捗: {progress:.1f}% ({batch_idx+1}/{num_batches} バッチ)', end='')
        except Exception as e:
            print(f'\n⚠️ バッチ{batch_idx+1}でエラー: {e}')
            continue
    
    print()  # 改行
    
    if predictions_list:
        forecast_result = pd.concat(predictions_list, ignore_index=True)
        
        elapsed = time.time() - start_time
        records_per_sec = total_records / elapsed if elapsed > 0 else 0
        
        print(f'\n✅ 予測完了:')
        print(f'  総件数: {len(forecast_result):,} 件')
        print(f'  実行時間: {elapsed:.1f}秒')
        print(f'  処理速度: {records_per_sec:,.0f} 件/秒')
        if USE_GPU:
            print(f'  💡 GPUにより高速化されました')
        
        # 予測値をわかりやすい列名に変更
        if 'prediction_label' in forecast_result.columns:
            forecast_result = forecast_result.rename(columns={
                'prediction_label': 'predicted_qty'  # 販売数量予測
            })
        
        # ========================================
        # 💰 売上金額の計算（予測数量 × 単価）
        # ========================================
        
        if 'predicted_qty' in forecast_result.columns and 'price' in forecast_result.columns:
            # 負の予測値を0に補正
            forecast_result['predicted_qty'] = forecast_result['predicted_qty'].clip(lower=0)
            
            # 売上金額 = 予測数量 × 単価
            forecast_result['predicted_sales_amt'] = (
                forecast_result['predicted_qty'] * forecast_result['price']
            ).round(0)
            
            print(f'\n💰 売上金額計算完了:')
            print(f'  予測販売数量合計: {forecast_result["predicted_qty"].sum():,.0f} 個')
            print(f'  予測売上金額合計: ¥{forecast_result["predicted_sales_amt"].sum():,.0f}')
        
        # ========================================
        # 📦 発注数量の計算（安全在庫係数）
        # ========================================
        
        SAFETY_FACTOR = 1.2  # 20%多めに発注（機会損失を防ぐ）
        
        if 'predicted_qty' in forecast_result.columns:
            forecast_result['order_qty'] = (
                forecast_result['predicted_qty'] * SAFETY_FACTOR
            ).round(0).astype(int)
            
            print(f'\n📦 発注数量計算完了:')
            print(f'  安全係数: {SAFETY_FACTOR}倍')
            print(f'  発注数量合計: {forecast_result["order_qty"].sum():,} 個')
            print(f'  （予測数量の{SAFETY_FACTOR}倍で在庫切れリスクを低減）')
        
        # ========================================
        # 📊 結果サマリー
        # ========================================
        
        print(f'\n📊 予測結果サマリー（上位10件）:')
        display_cols = []
        for col in ['日付', '店舗', '商品名', 'predicted_qty', 'order_qty', 'price', 'predicted_sales_amt']:
            if col in forecast_result.columns:
                display_cols.append(col)
        
        if display_cols:
            # 予測販売数量が多い順にソート
            top_results = forecast_result.nlargest(10, 'predicted_qty' if 'predicted_qty' in forecast_result.columns else 'predicted_sales_amt')
            print(top_results[display_cols])
        
        # 日別サマリー
        if '日付' in forecast_result.columns and 'predicted_qty' in forecast_result.columns:
            print(f'\n📅 日別サマリー:')
            daily_summary = forecast_result.groupby('日付').agg({
                'predicted_qty': 'sum',
                'order_qty': 'sum',
                'predicted_sales_amt': 'sum'
            }).reset_index()
            daily_summary.columns = ['日付', '予測販売数量', '発注数量', '予測売上金額']
            print(daily_summary)
        
        # 店舗別サマリー
        if '店舗' in forecast_result.columns and 'predicted_qty' in forecast_result.columns:
            print(f'\n🏪 店舗別サマリー（上位5店舗）:')
            store_summary = forecast_result.groupby('店舗').agg({
                'predicted_qty': 'sum',
                'order_qty': 'sum',
                'predicted_sales_amt': 'sum'
            }).reset_index()
            store_summary.columns = ['店舗ID', '予測販売数量', '発注数量', '予測売上金額']
            store_summary = store_summary.nlargest(5, '予測売上金額')
            print(store_summary)
        
        print(f'\n✅ 予測完了！次のステップ: Step 6-3でCSV出力してください')
    else:
        print('\n❌ 予測に失敗しました')


## 📊 ステップ7: 発注予測結果の読み方と実務での使い方

### 🎯 **このセクションの目的**
**上で出力された発注予測表を、明日からどう使うかを解説します。**

---

## 1️⃣ 出力されたCSVファイルの内容

### 📋 **メインファイル: `発注予測_2025-10-15_to_2025-10-21.csv`**

このファイルには、全商品×7日分の発注予測が含まれています。

| 列名 | 意味 | 使い方 |
|------|------|--------|
| **日付** | 販売予定日 | この日に店頭に並ぶ商品 |
| **曜日** | 曜日（Monday～Sunday） | 曜日別の発注パターン確認 |
| **店舗** | 店舗コード | 複数店舗の場合、店舗ごとに分ける |
| **商品コード** | SKU識別子 | 発注システムへの入力値 |
| **大分類/中分類/小分類** | カテゴリ | 発注作業の整理用 |
| **予測販売数量** | AIの予測値 | この数量が売れると予測 |
| **発注数量** | 実際の発注数 | **これを発注システムに入力** |
| **単価** | 平均単価 | 予算計算用 |
| **天気/気温** | 予測日の天候 | 判断材料 |
| **週末** | 週末/平日フラグ | 週末対応の目印 |

---

### 📅 **日付別ファイル: `発注予測_2025-10-15.csv`**

各日付ごとに分割されたファイルです。

**使い方**:
1. 毎朝、**その日の発注ファイル**を開く
2. **発注数量**列をそのまま発注システムに入力
3. カテゴリ別にソートして、効率的に発注作業を進める

---

### 🏪 **店舗別ファイル: `発注予測_店舗A_2025-10-15.csv`**

複数店舗がある場合、店舗ごとに分割されたファイルです。

**使い方**:
- 各店舗の店長に配布
- 店舗ごとの特性を反映した発注が可能

---

## 2️⃣ 発注作業の実務フロー（毎日のルーティン）

### 📝 **毎朝のチェックリスト**

#### **ステップ1: CSVファイルを開く（5分）**
```
1. output/発注予測_[今日の日付].csv を開く
2. Excelで開いて、カテゴリでソート
3. 発注システムと画面を並べる
```

#### **ステップ2: 発注数量を確認（5分）**
```
□ 発注数量が異常に多い商品はないか？
  → 通常の2倍以上 → 天気・イベントを再確認
  → 問題なければそのまま発注

□ 発注数量が0の商品はないか？
  → 0の場合 → 過去実績を確認して少量発注を検討
```

#### **ステップ3: 発注システムに入力（10～15分）**
```
1. カテゴリ別に発注数量を入力
2. 特に注意すべき商品:
   - 発注数量トップ20 → 欠品させない
   - 雨の日に売れる商品 → 降水確率を再確認
   - 週末に売れる商品 → 金曜の朝までに発注
```

#### **ステップ4: 調整（5分）**
```
□ 天気予報が変わっていないか確認
  → 変わっている場合、手動で±10%調整

□ 突発イベント（セールなど）がないか確認
  → ある場合、該当カテゴリを+20～30%増量

□ 昨日の売上実績と比較
  → 予測が大きく外れている場合、フィードバック
```

---

## 3️⃣ 結果の読み方（数字の意味）

### 📊 **日付別サマリーの見方**

上のセルで表示された「日付別サマリー」:

| 日付 | 発注数量 | 予測売上金額 | 商品種類数 |
|------|----------|------------|----------|
| 2025-10-15 | 850 | ¥127,500 | 120 |
| 2025-10-16 | 820 | ¥123,000 | 120 |
| 2025-10-19 (雨) | 950 | ¥142,500 | 120 |

**読み方**:
- **10/19（雨の日）** → 発注数量が通常より+15%増
- **意味**: AIが「雨の日は売上が増える」と判断
- **アクション**: 温かい商品・カップ麺などを重点発注

---

### 📦 **カテゴリ別サマリーの見方**

| 大分類 | 発注数量 | 予測売上金額 |
|--------|----------|------------|
| 弁当 | 2,100 | ¥315,000 |
| 飲料 | 1,800 | ¥108,000 |
| 総菜 | 1,500 | ¥225,000 |

**読み方**:
- **弁当が最も売れる** → 陳列スペースを最優先確保
- **発注数量の比率** → 棚割りの参考に

---

### 🔝 **発注数量トップ20の使い方**

上位20商品 = **絶対に欠品させてはいけない商品**

**実務アクション**:
1. **毎日チェック** → トップ20商品の在庫を毎夕確認
2. **フェース増** → 陳列スペースを通常の1.5倍確保
3. **前出し強化** → 昼・夕ピーク前に必ず前出し
4. **欠品時の対応** → 予備在庫を用意、または類似商品で代替

---

## 4️⃣ よくある質問（FAQ）

### ❓ **予測が外れた場合はどうする？**

**回答**:
- **1日だけ外れた** → 突発要因（競合店のセール等）の可能性。記録して次回に活かす
- **連続で外れる** → AIの再学習が必要。データ期間を延長（3ヶ月→6ヶ月）して再実行

---

### ❓ **安全係数はどう決める？**

**回答**:

| 状況 | 推奨値 | 理由 |
|------|--------|------|
| **通常運用** | 1.1～1.2 | 欠品と廃棄のバランス |
| **新商品** | 1.3～1.5 | 需要が読めないため保守的に |
| **廃棄率が高い** | 1.0～1.1 | 廃棄削減を優先 |
| **欠品が多発** | 1.2～1.3 | 売上機会損失を防ぐ |

**調整方法**:
- 1週間運用して、廃棄率と欠品率を測定
- 廃棄率5%以上 → 安全係数を-0.1下げる
- 欠品率3%以上 → 安全係数を+0.1上げる

---

### ❓ **天気予報が変わったらどうする？**

**回答**:
1. **前日夕方** に最新の天気予報を確認
2. **降水確率が30%以上変わった場合**:
   - 晴れ→雨 → 温かい商品を+10～20%増量
   - 雨→晴れ → 冷たい商品を+10～20%増量
3. **気温が5℃以上変わった場合**:
   - 暑くなる → 冷飲料・アイスを+20%増量
   - 寒くなる → ホット飲料・総菜を+20%増量

---

### ❓ **発注数量が0の商品はどうする？**

**回答**:
- **AIの判断**: その商品はその日売れにくい
- **実務対応**:
  1. 過去実績を確認 → 本当に売れていないか？
  2. 売れている → 最低発注数（1～2個）を手動入力
  3. 売れていない → 0のまま発注しない（在庫削減）

---

## 5️⃣ 効果測定（1週間後にチェック）

### ✅ **成功指標**

| 指標 | 目標値 | 測定方法 |
|------|--------|----------|
| **欠品率** | 3%以下 | 欠品商品数 ÷ 全商品数 |
| **廃棄率** | 5%以下 | 廃棄数量 ÷ 発注数量 |
| **予測精度** | 80%以上 | 実売数量 ÷ 予測数量 |
| **売上増加率** | +5%以上 | 今週売上 vs 先週売上 |

**測定シート（Excelで作成）**:
```
| 日付 | 発注数量 | 実売数量 | 廃棄数量 | 欠品商品数 | 予測精度 |
|------|----------|----------|----------|-----------|---------|
| 10/15 | 850 | 820 | 30 | 2 | 96% |
| 10/16 | 820 | 790 | 30 | 1 | 96% |
| ...  | ... | ... | ... | ... | ... |
```

---

## 📌 まとめ: AI発注予測の活用ポイント

### **最重要ポイント 3つ**

1. **毎朝5分のルーティン化**
   - CSVファイルを開く → 発注数量を確認 → 発注システムに入力

2. **トップ20商品は絶対欠品させない**
   - 毎夕、在庫チェック
   - フェース1.5倍確保
   - 予備在庫を用意

3. **1週間運用して、安全係数を調整**
   - 廃棄率5%以上 → 係数-0.1
   - 欠品率3%以上 → 係数+0.1

---

**これで、AI発注予測システムの説明は完了です！**

明日から実際に運用してみてください。1週間後に効果を測定し、継続的に改善していきましょう。

In [ ]:
# ========================================# 💾 Step 6-3: 発注表をCSVファイルに出力# ========================================from pathlib import Pathif 'forecast_result' not in globals() or forecast_result.empty:    print('[ERROR] 予測結果が見つかりません。先にStep 6-2を実行してください。')else:    # outputディレクトリ作成    output_dir = Path('output')    output_dir.mkdir(exist_ok=True)        print('\n' + '='*60)    print('💾 発注表CSVファイルを出力中...')    print('='*60)        # ========================================    # 📄 メイン発注表（全データ）    # ========================================        # 日付範囲を取得    if '日付' in forecast_result.columns:        # Categorical型の場合は文字列に変換        date_col = forecast_result['日付']        if hasattr(date_col, 'cat'):  # Categoricalの場合            date_col = date_col.astype(str)                # ソートして最初と最後を取得        dates_sorted = sorted(date_col.unique())        start_date = dates_sorted[0] if len(dates_sorted) > 0 else 'unknown'        end_date = dates_sorted[-1] if len(dates_sorted) > 0 else 'unknown'        output_filename = f'発注表_全店舗_{start_date}_to_{end_date}.csv'    else:        output_filename = '発注表_全店舗.csv'        output_path = output_dir / output_filename        # 発注表として必要な列を選択    order_cols = []    for col in ['日付', '店舗', '商品名', 'predicted_qty', 'order_qty', 'price', 'predicted_sales_amt']:        if col in forecast_result.columns:            order_cols.append(col)        # 日本語列名に変換    order_df = forecast_result[order_cols].copy()    rename_dict = {        '日付': '日付',        '店舗': '店舗ID',        '商品名': '商品ID',        'predicted_qty': '予測販売数量',        'order_qty': '発注数量',        'price': '単価',        'predicted_sales_amt': '予測売上金額'    }    order_df = order_df.rename(columns={k: v for k, v in rename_dict.items() if k in order_df.columns})        # CSV出力    order_df.to_csv(output_path, index=False, encoding='utf-8-sig')        print(f'\n✅ メイン発注表を保存:')    print(f'  ファイル名: {output_path}')    print(f'  ファイルサイズ: {output_path.stat().st_size / 1024:.1f} KB')    print(f'  レコード数: {len(order_df):,} 件')        # ========================================    # 📅 日付別CSVファイル    # ========================================        if '日付' in order_df.columns:        print(f'\n📅 日付別CSVファイルを出力中...')        for date in sorted(order_df['日付'].unique()):            date_df = order_df[order_df['日付'] == date].copy()            date_filename = f'発注表_{date}.csv'            date_path = output_dir / date_filename            date_df.to_csv(date_path, index=False, encoding='utf-8-sig')                        total_qty = date_df['発注数量'].sum() if '発注数量' in date_df.columns else 0            total_amt = date_df['予測売上金額'].sum() if '予測売上金額' in date_df.columns else 0                        print(f'  ✅ {date_filename}: {len(date_df):,}件 | 発注{total_qty:,}個 | 予測売上¥{total_amt:,.0f}')        # ========================================    # 🏪 店舗別CSVファイル    # ========================================        if '店舗ID' in order_df.columns and order_df['店舗ID'].nunique() > 1:        print(f'\n🏪 店舗別CSVファイルを出力中...')        for store_id in sorted(order_df['店舗ID'].unique()):            store_df = order_df[order_df['店舗ID'] == store_id].copy()            store_filename = f'発注表_店舗{int(store_id)}.csv'            store_path = output_dir / store_filename            store_df.to_csv(store_path, index=False, encoding='utf-8-sig')                        total_qty = store_df['発注数量'].sum() if '発注数量' in store_df.columns else 0            total_amt = store_df['予測売上金額'].sum() if '予測売上金額' in store_df.columns else 0                        print(f'  ✅ {store_filename}: {len(store_df):,}件 | 発注{total_qty:,}個 | 予測売上¥{total_amt:,.0f}')        # ========================================    # 🛍️ 商品別サマリー（高回転商品TOP100）    # ========================================        if '商品ID' in order_df.columns and '発注数量' in order_df.columns:        print(f'\n🛍️ 商品別サマリーを出力中...')                sku_summary = order_df.groupby('商品ID').agg({            '発注数量': 'sum',            '予測売上金額': 'sum'        }).reset_index()        sku_summary.columns = ['商品ID', '総発注数量', '総予測売上金額']        sku_summary = sku_summary.nlargest(100, '総発注数量')                sku_summary_path = output_dir / '商品別サマリー_TOP100.csv'        sku_summary.to_csv(sku_summary_path, index=False, encoding='utf-8-sig')        print(f'  ✅ 商品別サマリー_TOP100.csv: {len(sku_summary)}件')        # ========================================    # 📊 全体サマリーレポート    # ========================================        print(f'\n📊 全体サマリー:')    if '発注数量' in order_df.columns:        print(f'  総発注数量: {order_df["発注数量"].sum():,} 個')    if '予測売上金額' in order_df.columns:        print(f'  総予測売上: ¥{order_df["予測売上金額"].sum():,.0f}')    if '店舗ID' in order_df.columns:        print(f'  対象店舗数: {order_df["店舗ID"].nunique()} 店舗')    if '商品ID' in order_df.columns:        print(f'  対象商品数: {order_df["商品ID"].nunique()} SKU')    if '日付' in order_df.columns:        # 日付列をdatetimeに変換してmin/maxを取得        try:            date_col = pd.to_datetime(order_df["日付"])            print(f'  予測期間: {date_col.min()} ～ {date_col.max()}')        except:            # Categorical型の場合はユニーク値でソート            dates = sorted(order_df["日付"].unique())            if len(dates) > 0:                print(f'  予測期間: {dates[0]} ～ {dates[-1]}')        print(f'\n✅ すべてのCSVファイルを output/ ディレクトリに保存しました')    print(f'\n💡 発注表の使い方:')    print(f'  1. 日付別CSV: 日次の発注計画に使用')    print(f'  2. 店舗別CSV: 店舗ごとの発注指示に使用')    print(f'  3. 商品別サマリー: 重点商品の在庫管理に使用')
